In [1]:
# Import of important libraries in Python:
import sys, os, time, importlib, glob, pdb
import matplotlib, pylab as pl, numpy as np
from numpy import log10 as log
import h5py, astropy, scipy, scipy.stats
from astropy import units as un, constants as cons
from scipy import interpolate
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits import mplot3d
import statistics as s
import random
from os import listdir
from os.path import isfile, join
import MyTools.AnalysisTools as l
import MyTools.HalosData as HD
import MyTools.Verdict as Verdict
from MyTools.Project3MgIIAnalysisConfigs import AllGasAverageDensityAgainstRedshiftConfig as c
import math
import matplotlib.patches as mpatches
from astropy import units as un, constants as cons
import pylab as pl
import seaborn as sns
import palettable
#%matplotlib auto
###################################################################################################################################
# FIRE studio libraries
from OldFireStudio.firestudio.studios.gas_studio import GasStudio
from OldFireStudio.firestudio.studios.star_studio import StarStudio
###################################################################################################################################
# Important constants
X=0.7 # Hydrogen mass fraction
h=0.7 # Hubble parameter in units of 100 km/s Mpc^-1
kpc_3Tocm_3Factor=2.94e64 # kpc^3 to cm^3 multiply factor
ProtonMassInSunMass=8.4e-58 # Proton mass in solar mass
###################################################################################################################################
# Print the config file:
SimulationName=c.SimulationName
print("Simulation name:",SimulationName)
print("Resolution:",c.Resolution)
print("Snapshot numbers:",c.SnapshotsNumbers)

Couldn't import numba. Missing:
abg_python.all_utils.get_cylindrical_velocities
don't have phil's colormaps
Simulation name: m12f
Resolution: _r7100
Snapshot numbers: ['500', '495', '490', '485', '480', '475', '470', '465', '460', '455', '450', '445', '440', '435', '430', '425', '420', '415', '410', '405', '400', '395', '390', '385', '380', '375', '370', '365', '360', '355', '350', '345', '340', '335', '330', '325', '320', '315', '310', '305', '300', '295', '290', '285', '280', '275', '270', '265', '260', '255', '250', '245', '240', '235', '230', '225', '220', '215', '210', '205', '200', '195', '190', '185', '180', '175', '170']


In [2]:
# Creates the full paths to the simulation snapshots
ListOfStampedeSimulationsPaths=[]
SnapDirect=c.StampedeSnapDirect
SnapDirect=SnapDirect+c.SimulationName+c.Resolution
ListOfStampedeSimulationsPaths.append(SnapDirect)
###################################################################################################################################
ListOfOutputFolderPathsForStampede=[]
for i in ListOfStampedeSimulationsPaths:
    temp=i+"/output"
    ListOfOutputFolderPathsForStampede.append(temp)
ListOfFullPathsForStampede=[]
for i in ListOfOutputFolderPathsForStampede:
    OutputFiles=os.listdir(i)
    for j in c.SnapshotsNumbers:
        option1="snapshot_"+j+".hdf5"
        option2="snapdir_"+j
        if(option1 in OutputFiles):
            FullPath=i+"/"+option1
        if(option2 in OutputFiles):
            FullPath=i+"/"+option2
        ListOfFullPathsForStampede.append(FullPath) 
print("Full paths of the simulation snapshots:")
print(ListOfFullPathsForStampede)

Full paths of the simulation snapshots:
['/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_500', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_495', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_490', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_485', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_480', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_475', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_470', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_465', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_460', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapdir_455', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12f_r7100/output/snapd

In [ ]:
for SnapDirectIndex in range(0,len(ListOfFullPathsForStampede)):
    SnapDirect=ListOfFullPathsForStampede[SnapDirectIndex]
    SnapNumber=c.SnapshotsNumbers[SnapDirectIndex]
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append only hdf5 files to the snapshots parts list
                
    print("############################################################################################################################")
    print("Starting snapshot number:",SnapNumber)           
    z=HD.Redshift(SimulationName,int(c.SnapshotsNumbers[SnapDirectIndex]))
    Mvir=HD.HaloMvir(SimulationName,z)
    Rvir=HD.HaloRvir(SimulationName,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationName,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationName,z)
    afactor=HD.HaloaFactor(SimulationName,z)

    # This part intended to merge all the data from the different snapshot parts (if there are more than 1 part)
    GasMasses=[]
    GasDensities=[]
    GasRadiuses=[]
    GasTemperatures=[]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, real center coordinates, real center velocity

        GasMassesOfOneSnapshot=Snap1.MassesInSunMasses(0)# The masses of the gas particles in [M_sun]
        GasMasses.extend(GasMassesOfOneSnapshot)
        
        GasDensityOfOneSnapShot=Snap1.DensityIn_gr_cm_3(0)# The densities of the gas particles in [gr/cm^3]
        GasDensities.extend(GasDensityOfOneSnapShot)
        
        GasRadiusesOfOneSnapShot=Snap1.RadiusesInkpc(0)# The radius of the gas particles in [kpc]
        GasRadiuses.extend(GasRadiusesOfOneSnapShot)

        GasTemperaturesOfOneSnapShot=Snap1.TemperaturesInK()# The temperature of the gas particles in [K]
        GasTemperatures.extend(GasTemperaturesOfOneSnapShot)
            
    GasMasses=np.array(GasMasses)
    GasDensities=np.array(GasDensities)
    GasRadiuses=np.array(GasRadiuses)
    GasTemperatures=np.array(GasTemperatures)
    
    ListOfRvirDivide=np.array([0.1*Rvir,0.2*Rvir,0.3*Rvir,0.4*Rvir,0.5*Rvir,0.6*Rvir,0.7*Rvir,0.8*Rvir,0.9*Rvir,Rvir])
    ListOfMassRhoSummationsForAllRvirDivides=[0,0,0,0,0,0,0,0,0,0]
    ListOfMassLogRhoSummationsForAllRvirDivides=[0,0,0,0,0,0,0,0,0,0]
    ListOfMassSummationsForAllRvirDivides=[0,0,0,0,0,0,0,0,0,0]
    ListOfParticlesCountInEachRvirDivides=[0,0,0,0,0,0,0,0,0,0]
    Epsilon=0.01*Rvir
    for i in range(0,len(GasTemperatures)):
        M=GasMasses[i]
        Rho=GasDensities[i]
        R=GasRadiuses[i]
        T=GasTemperatures[i]
        for RvirDivideIndex in range(0,len(ListOfRvirDivide)):
            RvirDivide=ListOfRvirDivide[RvirDivideIndex]
            if(R>(RvirDivide-Epsilon) and R<(RvirDivide+Epsilon)):
                ListOfMassRhoSummationsForAllRvirDivides[RvirDivideIndex]=ListOfMassRhoSummationsForAllRvirDivides[RvirDivideIndex]+M*Rho
                ListOfMassLogRhoSummationsForAllRvirDivides[RvirDivideIndex]=ListOfMassLogRhoSummationsForAllRvirDivides[RvirDivideIndex]+M*np.log10(Rho)
                ListOfMassSummationsForAllRvirDivides[RvirDivideIndex]=ListOfMassSummationsForAllRvirDivides[RvirDivideIndex]+M
                ListOfParticlesCountInEachRvirDivides[RvirDivideIndex]=ListOfParticlesCountInEachRvirDivides[RvirDivideIndex]+1
    
    ListOfMassRhoSummationsForAllRvirDivides=np.array(ListOfMassRhoSummationsForAllRvirDivides)
    ListOfMassLogRhoSummationsForAllRvirDivides=np.array(ListOfMassLogRhoSummationsForAllRvirDivides)
    ListOfMassSummationsForAllRvirDivides=np.array(ListOfMassSummationsForAllRvirDivides)
    ListOfParticlesCountInEachRvirDivides=np.array(ListOfParticlesCountInEachRvirDivides)
    
    AverageDensityForAllRvirDivides=ListOfMassRhoSummationsForAllRvirDivides/ListOfMassSummationsForAllRvirDivides
    AverageLogDensityForAllRvirDivides=ListOfMassLogRhoSummationsForAllRvirDivides/ListOfMassSummationsForAllRvirDivides
                               #     0     , 1,  2  ,  3  ,               4                ,                 5
    DataForSpecificSnapshot=[] # SnapNumber, z, Mvir, Rvir, AverageDensityForAllRvirDivides, AverageLogDensityForAllRvirDivides
    DataForSpecificSnapshot.append(SnapNumber)
    DataForSpecificSnapshot.append(z)
    DataForSpecificSnapshot.append(Mvir)
    DataForSpecificSnapshot.append(Rvir)
    DataForSpecificSnapshot.append(AverageDensityForAllRvirDivides)
    DataForSpecificSnapshot.append(AverageLogDensityForAllRvirDivides)
    DataForSpecificSnapshot=np.array(DataForSpecificSnapshot,dtype=object)
    np.savez_compressed('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/OutpotsMgII-AllGasAverageDensityAgainstRedshift/'+SimulationName+'/'+str(SimulationName)+'_'+str(SnapNumber)+'.npz',DataForSpecificSnapshot,dtype=float)
    
    print("Done snapshot number:",SnapNumber)
print("Done all snapshots in this simulation.")
print("############################################################################################################################")
print("############################################################################################################################")
print("############################################################################################################################")

############################################################################################################################
Starting snapshot number: 500
Redshift: 0.16946
M_vir: 1.10e+12
R_vir: 192.13141107861748
Center coordinates: [27422.67696244, 33193.71684617, 32666.91460904]
Center velocity: [-152.95, 165.5, 115.52]
a factor: 0.8550955141689327
loaded PartType0, Masses
loaded PartType0, Density
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Masses
loaded PartType0, Density
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Masses
loaded PartType0, Density
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Masses
loaded PartType0, Density
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
Done snapshot number: 500
############################################################################################################################
Starting snapshot number: 495
Redshift

In [ ]:
from IPython.lib.display import Audio

framerate=4410
play_time_seconds=3

t=np.linspace(0,play_time_seconds,framerate*play_time_seconds)
audio_data=np.sin(2*np.pi*300*t)+np.sin(2*np.pi*240*t)
Audio(audio_data,rate=framerate,autoplay=True)

In [ ]:
Directory='/home1/08289/tg875885/radial_to_rotating_flows/Aharon/OutpotsMgII-CoolGasAverageDensityAgainstRedshift/m12i/600.npz'
DataLoad=np.load(Directory,allow_pickle=True)['arr_0']
print(DataLoad)

In [ ]:
lst=[]
for i in range(600,171,-5):
    lst.append(str(i))
print(lst)
print(len(lst))